This Notebook is a template to extract one-dimensional cell migration data from a full experiment

In [10]:
##Import all necessary libraries
import sys
import os
import time
sys.path.append('/project/ag-moonraedler/MAtienza/celltracker')
#sys.path.append('/home/m/Miguel.Atienza/celltracker')
from celltracker.pipeline import Track
from celltracker.pipeline import run_pipeline
from nd2reader import ND2Reader
from datetime import datetime
from celltracker.database import Database
import numpy as np
from viewer import notebook_viewer as nbv
import matplotlib.pyplot as plt
from tqdm import tqdm
import logging
import traceback
from skimage.io import imread
%matplotlib ipympl
%load_ext autoreload
%autoreload

In [11]:
#Path to folder in which the experiment is contained
data_path = './'

#Name of main time-lapse experiment file 
nd2_file = 'timelapse.nd2'

#Name of file containing patterns
lanes_file = 'pattern.nd2'

#Path to folder where results should be output (Make sure that this directory exists before running the pipeline)
path_out = './extraction/'

#Path to database
db_path = '/project/ag-moonraedler/MAtienza/database/onedcellmigration.db'

In [ ]:
#Check and optimise the lane detection
laneviewer = nbv.LaneViewer(data_path+lanes_file)

In [4]:
#Save optimal values
lane_distance = laneviewer.ld.value
lane_low_clip, lane_high_clip = laneviewer.clip.value
plt.close('all')

In [3]:
#Check trackpy and optimise
tpviewer = nbv.TpViewer(data_path+nd2_file)
#batch in blue and link in red

GridspecLayout(children=(Canvas(header_visible=False, layout=Layout(grid_area='widget001'), toolbar=Toolbar(to…

In [ ]:
min_mass, max_travel, track_memory, diameter, min_frames = tpviewer.min_mass.value, tpviewer.max_travel.value, tpviewer.track_memory.value, tpviewer.diameter.value, tpviewer.min_frames.value

In [6]:
#Now check and optimise cellpose
cellposeviewer = nbv.CellposeViewer(data_path+nd2_file)

GridspecLayout(children=(Canvas(header_visible=False, layout=Layout(grid_area='widget001'), toolbar=Toolbar(to…

In [4]:
cyto_diameter=cellposeviewer.diameter.value
flow_threshold=cellposeviewer.flow_threshold.value
mask_threshold=cellposeviewer.mask_threshold.value
normalize=True

In [5]:
##Access metadata from nd2 file
f = ND2Reader(os.path.join(data_path, nd2_file))
metadata = f.metadata

In [8]:
#Fill in data about experiment
Experiment_data = {
    #'Experiment_id':,
    'Path':data_path,
    'Date': metadata['date'].strftime('%d-%m-%Y %H:%m'),
    'celltype': 'MDA-MB-231',
    'microscope': 'UNikon',
    'nframes': f.sizes['t'],
    'nfov': f.sizes['v'],
    'channels': str(metadata['channels']),
    'fpm': f.sizes['t']/(1e-3*(f.metadata['events'][-1]['time'] - f.metadata['events'][0]['time'])/60),
    'pixels/um': 1.538,
    'bitsperpixel': 16,
    'Author': 'Miguel Atienza'}

In [9]:
1/Experiment_data['fpm']

2.5614553594607625

In [10]:
#Initiate a Database object to access and load data into the database
db = Database(db_path)
#experiment_id = db.add_experiment(Experiment_data)
experiment_id=6
print(experiment_id)

6


In [56]:
#Get the number of fields of view in the experiment and start a timer
nfov = f.sizes['v']
start_time = time.time()
fovs = list(range(0, nfov))

In [12]:
#Choose whether results should be saved to sql database or not
sql=False

In [ ]:
##You can change the arguments to use values from calibration and etc...
run_pipeline(data_path, nd2_file, lanes_file, path_out, frame_indices, manual=False, fovs=None, sql=False, lane_distance=30,
lane_low_clip=0, lane_high_clip=2000, min_mass=2.65e5, max_travel=15, track_memory=15, diameter=15, min_frames=10, cyto_diameter=29, 
flow_threshold=1.25, mask_threshold=0, pretrained_model='mdamb231', use_existing_parameters=False)